# BlockEncoding class 201

Welcome back! In the previous tutorial, we learned how to "embed" a non-unitary matrix $A$ in the top left block a larger unitary $U$ using the ``BlockEncoding`` class. But just having $A$ block encoded usually isn't enough. Often, we want to compute functions of that matrix—like $e^{-iAt}$ for simulating physics (Hamiltonians), or $A^{-1}$ for solving linear systems, or even applying step-functions or the gaussian as a filter for ground-state preparation. 

Classically, if you have a number $x$, you can calculate $f(x)$ using a Taylor series or a polynomial approximation. In quantum computing, we can actually do the same thing by using an approach called Quantum Signal Processing (QSP). This will be covered towards the end of this tutorial. To get there, we first need to learn how to use the BlockEncoding class to transform into a "Quantum Walk" via Qubitization.

After learning about Qubitization, this tutorial will also explain how to block-encode Chebyshev polynomials and why they are useful. We will then show how to run the algorithms stemming from QSP in Qrisp, and how the ``BlockEncoding`` class makes polynomial transformations, solving linear systems, and hamiltonian simulation as simple as calling ``.poly``, ``.inv``, and ``.sim`` methods respectively.

But first thing's first. Let's break down the concept called qubitization.

## Qubitization
If a Block Encoding is a "static snapshot" of a matrix $A$, Qubitization is what makes it "move". Technically, Qubitization is a method to transform an $(\alpha, m, \epsilon)$-block-encoding of a matrix $A$ into a special unitary operator $W$, often refferod to as the "walk operator". This operator has a nice property: it maps the eigenvalues $\lambda$ of $A$ to the eigenvalues $e^{\pm i \arccos(\lambda/\alpha)}$ in a set of two-dimensional invariant subspaces. 

Given a Hermitian matrix $H$ and its block-encoding $(U, G)$, where $G\ket{0} = \ket{G}$, we use the definition of the reflection operator $R$ acting on the ancilla space as $R = (2\ket{G}\bra{G}_a \otimes \mathbb{1}_a)\otimes \mathbb{1}_{s}$ from [Lemma 1 in Exact and efficient Lanczos method on a quantum computer](https://arxiv.org/pdf/2208.00567). To "qubitize" the encoding, we interleave the SELECT operator with this reflection. The Qubitized Walk Operator $W$ is defined as $W = \text{SELECT}\cdot R$.

Rigorous analysis (see Lin Lin, Chapter 8) shows that if $\ket{\psi_\lambda}$ is an eigenvector of $H/\alpha$ with eigenvalue $\lambda \in [-1, 1]$, the operator $W$ acts on a 2D subspace spanned by $\ket{G}\ket{\psi_\lambda}$ and its orthogonal complement $\ket{\perp}$. Within this subspace, the eigenvalues of $W$ are $\mu_\pm = \lambda \pm i\sqrt{1-\lambda^2} = e^{\pm i \arccos(\lambda)}$. Essentially, Qubitization "lifts" the eigenvalues of our matrix onto the unit circle in the complex plane, allowing us to manipulate them using phase rotations.

This seems like a mouthful, but in order for this tutorial to cater to both developers coming from the classical domain, as well as researchers in quantum computing, it's the "necessary evil". To make it up to you, we're going to show how in Qrisp, you don't need to build these reflections manually. But first, some visual aid so that you see that it's not as complex as it sounds.

![Alt text](walk_operator.png)

In the previous tutorial you've already learned how we use the $\text{SELECT}$ by just calling ``q_switch``. Well, what if we told you that performing the reflection operation above you can just use the ``reflection`` function? Yup, that's the cool thing about modular software development approach Qrisp is taking with its focus on high-level abstractions. Let's get to coding!

### Qubitization in Qrisp as ``.qubitization``

While understanding the internal mechanics of q_switch and reflection is valuable for intuition, Qrisp abstracts this complexity away for standard operations. The BlockEncoding class features a dedicated method, .qubitization(), which automatically constructs the walk operator $W$ from your input matrix.This method handles the heavy lifting: it identifies the necessary reflection operators $R$ and interleaves them with the signal oracle (the block-encoding unitary $U$). If the original block-encoding unitary $U$ isn't Hermitian (i.e., $U^2 \neq \mathbb{1}$), Qrisp automatically handles the Hermitian embedding—often requiring one additional ancilla qubit—to ensure the walk operator remains unitary.Here is how you can transform a Hamiltonian into its qubitized walk operator in just a few lines:

In [1]:
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y, Z

# 1. Define a Hamiltonian
# We create a simple Hamiltonian H = X_0 Y_1 + 0.5 Z_0 X_1
H = X(0)*Y(1) + 0.5*Z(0)*X(1)

# 2. Create the initial Block Encoding
# This generates the (U, G) pair discussed above
BE = BlockEncoding.from_operator(H)

# 3. Generate the Qubitized Walk Operator
# This automates the construction of W = SELECT * R
BE_walk = BE.qubitization()

The resulting object, ``BE_walk``, is a new ``BlockEncoding`` instance representing the walk operator $W$. A thing to remember from this example is the fact that when you invoke methods of the ``BlockEncoding`` class like ``.qubitization``, or later ``.poly`` and ``.sim``, Qrisp qubitizes your BlockEncoding object under the hood, handling the ancilla management and reflection logic for you, abstracting away the need to know how to implement these methods as (trigger warning) circuits.

Crucially, the ``.qubitization`` operator is also how oneencodes the Chebyshev polynomials of the Hamiltonian, as we'll learn in the next part of the tutorial. 

### Block encoding Chebyshev polynomials
One of the most powerful features of Qubitization is its natural relationship with Chebyshev polynomials of the first kind, $T_k(x)$, defined as $T_k(\cos \theta) = \cos(k\theta)$. If we apply the walk operator $W$ $k$-times, the resulting unitary $W^k$ contains $T_k(\frac{A}{\alpha})$ block encoded in the top-left block:
$$(\bra{G} \otimes \mathbb{1}) W^k (\ket{G} \otimes \mathbb{1}) = T_k(\frac{A}{\alpha}).$$

"But what's so special about Chebyshev polynomials", you might be wondering. As noted in Lin Lin’s lecture notes, Chebyshev polynomials are "optimal" in two senses:

- Iterative Efficiency: Because $W$ is a single unitary, applying $W^k$ requires only $k$ queries to the block encoding. This is much cheaper than the $O(2^n)$ terms often required by naive Taylor series expansions.

- Approximation Theory: According to the Chebyshev Equioscillation Theorem, $T_k(x)$ provides the best uniform approximation to a function over the interval $[-1, 1]$. This ensures that our quantum algorithm achieves the desired precision $\epsilon$ with the minimum possible quantum resources.

we think that, again, some visual aid is needed. By appyling the ``.qubitization`` operator $k$ times, we block encode the $k$-th Chebyshev polynomial of the first kind $T_k$. If you apply $W^k=(RU)^k$ $k$ times, you get $T_k$ block encoded. Do it once, $k=1$, you get the top left figure. Do it twice ($k=2$), you block-encode $T_2$ (top right figure). Do it $k=5$ times... yup, you guessed it (bottom right figure):

![Alt text](chebyshev.png)

Just as with the basic walk operator, Qrisp abstracts the iterative application of $W$ into a simple method call. The ``BlockEncoding`` class provides a ``.chebyshev(k)`` method, which returns a new block encoding for the $k$-th Chebyshev polynomial $T_k$. This handles the construction of $W^k$ (or the appropriate sequence of reflections and select/``qswitch`` operators) internally.

Here is how to generate and apply a Chebyshev polynomial transformation to a Hamiltonian:

In [2]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y, Z

# 1. Define the Hamiltonian
H = X(0)*X(1) + 0.5*Z(0)*Z(1)

# 2. Create the initial Block Encoding
BE = BlockEncoding.from_operator(H)

# 3. Create the Block Encoding for T_2(H)
# This generates the circuit for the 2nd order Chebyshev polynomial
BE_cheb_2 = BE.chebyshev(k=2)

# 4. Use the new Block Encoding
# For example, applying it via Repeat-Until-Success (RUS) to a state
def operand_prep():
    # Prepare an initial state, e.g., uniform superposition on 2 qubits
    qv = QuantumFloat(2)
    h(qv)
    return qv

@terminal_sampling
def main(BE):
    # Apply the block encoded operator T_2(H) to the state
    qv = BE.apply_rus(operand_prep)()
    return qv

# Execute
result = main(BE_cheb_2)
print(result)

{0.0: 0.4900000153295707, 3.0: 0.48999992592259706, 1.0: 0.010000029373916136, 2.0: 0.010000029373916136}


The ``.chebyshev(k)`` method is particularly useful for building polynomial approximations where $T_k$ terms are the basis functions. By default (``rescale=True``), it returns a block-encoding of $T_k(H)$, managing the normalization factors via Quantum Eigenvalue Transformation (QET) logic. If you need the raw polynomial $T_k(H/\alpha)$ relative to the block-encoding's normalization $\alpha$, you can set ``rescale=False``.

As learned in the previous tutorial you can also perform resource analysis by just calling the ``.resources`` method.

In [6]:
cheb_resources = BE_cheb_2.resources(operand_prep)()
print(cheb_resources)

AttributeError: 'function' object has no attribute 'template'

Let's, at this point, also show how we could do a quick benchmark of the scaling of resources needed. Since the walk operatore $W=(RU)$ is exactly the block encoding overhead for Chebyshev polynomials, we can see how the resources scale with repeated applications of them to block-encode $T_k$.

In [7]:
for k in range(1, 8):
    # Generate the k-th Chebyshev Block Encoding
    # We use rescale=False to look at the raw complexity of the walk operator iterations
    BE_cheb = BE.chebyshev(k, rescale=False)
    
    # Extract resource dictionary
    cheb_resources = BE_cheb.resources(operand_prep)()
    print(f"k = {k}: {cheb_resources}")

AttributeError: 'function' object has no attribute 'template'

If you look at the printed output of your benchmark, you’ll notice a very satisfying trend: the gate counts and depth grow linearly with $k$.

In the classical world, high-order polynomial approximations often come with a heavy computational tax. On a quantum computer, thanks to Qubitization, the $k$-th Chebyshev polynomial $T_k$ is implemented simply by repeating the walk operator $W$ exactly $k$ times. This efficiency is the "secret sauce" behind many modern quantum algorithms—we get high-precision approximations without the exponential gate-count explosion.

### Quantum Lanczos method

The ability to efficiently block-encode Chebyshev polynomials isn't just a mathematical flex; it’s a prerequisite for one of the most exciting algorithms in recent years: the Quantum Lanczos Method.

As detailed in the paper [Exact and efficient Lanczos method on a quantum computer](https://arxiv.org/pdf/2208.00567), these polynomials are used to construct what is known as a Krylov subspace. By applying different orders of $T_k$ to an initial state, we can "scan" the spectrum of a Hamiltonian.

To put it in a more digestible way, the Lanczos method projects the Hamiltonian into a much smaller, manageable subspace. We then calculate the overlaps (matrix elements) between these Chebyshev-transformed states.

If you're wondering why you should care, this allows for highly accurate Ground State Preparation! By diagonalizing the matrix in this small Krylov subspace, we can find the lowest eigenvalue (the ground state energy) and the corresponding state with far fewer resources than traditional Phase Estimation. 

In Qrisp, the ``.chebyshev`` method serves as the engine for these advanced spectral methods. If you're looking to dive deep into the implementation of this, you can check out the ``lanczos_method`` reference page in our documentation. To provide a glimpse, here is one short example and a comparison to the classical solution to verify the result:

In [ ]:
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()

While Chebyshev polynomials are the "optimal" choice for many tasks, they are still just one type of polynomial. What if you want to implement a step function to filter states? Or an inverse function $1/x$ for solving linear systems of equations? Or a complex exponential $e^{-ixt}$ for Hamiltonian simulation?

To do that, we need a more generalized framework that treats the walk operator not just as a repeating block, but as a tunable sequence. This brings us to the "Grand Unified Theory" of quantum algorithms: Quantum Signal Processing (QSP).

In the next chapter, we’ll see how Qrisp takes everything we've learned about block encodings and qubitization to let you implement an arbitrary polynomial transformation by simply calling ``.poly()``.

#### Inverse using ``.inv``

EXPLAIN

In [ ]:
### EXAMPLE

#### Polynomial transformations using ``.poly``

EXPLAIN

In [ ]:
### EXAMPLE

#### Hamiltnian simulation using ``.sim``

EXPLAIN

In [ ]:
### EXAMPLE